### :snowflake: College of AI: Data Agent HOL Snowflake Setup Guide :snowflake:

This notebook will provision the roles, service user, compute resources, and structured data needed to complete the Data Agent hands-on lab using Snowflake Cortex Analyst, Search and Agents.

🧪 **HOL Setup Overview**
- Establish a **dedicated service user** (`dataagent_user`) to run Cortex workloads
- Provide a **custom warehouse and database** (`AICOLLEGE`) for isolation
- Create a **stage** (`TRANSCRIPTS`) for uploading transcripts and semantic YAML for analyst use
- Grant **read and compute privileges** to your `AICOLLEGE` role
- Enable **cross-region Cortex inference** (required for many Cortex features)
- Generate a **PAT** and configure **key-pair authentication** for secure service access via **Slack**
- Load **mock customer data** into tables used by Cortex Analyst:
    - `CUSTOMER_MEETINGS`
    - `CUSTOMER_MEETING_OUTCOMES`
    - `CUSTOMER_INTERACTIONS`
    - `CUSTOMER_INDUSTRY`
-Create a unified view called `CUSTOMER_INSIGHTS` that merges meetings and interactions into a single analytics layer

🔧 **Required Manual Updates**
Before running the full notebook, make sure to update the following cells:

|🔢 Step|🔄 What to Update|📌 Notes|
|-------|-----------------|--------|
|SQL_2|`<YOUR_USERNAME>`|Replace with your Snowflake login username to give yourself access to the AICOLLEGE role
|SQL_4|Copy PAT Token|After generating the PAT, immediately copy the token shown in the message box — it cannot be retrieved later
|SQL_5|RSA_PUBLIC_KEY|Paste your RSA public key used for key-pair authentication

In [ ]:
import streamlit as st
import pandas as pd
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In [ ]:
-- Create a dedicated role & service user
USE ROLE ACCOUNTADMIN;

CREATE ROLE IF NOT EXISTS aicollege;

CREATE OR REPLACE USER dataagent_user
  TYPE = SERVICE
  DEFAULT_ROLE = aicollege
  COMMENT = 'Service user for Data Agent HOL';

GRANT ROLE aicollege TO USER dataagent_user;

In [ ]:
-- also grant yourself so you can inspect
GRANT ROLE aicollege TO USER <YOUR_USERNAME>;

In [ ]:
-- Create database & warehouse
CREATE DATABASE IF NOT EXISTS aicollege;

CREATE WAREHOUSE IF NOT EXISTS aicollege
  WITH WAREHOUSE_SIZE = 'XSMALL'
  AUTO_SUSPEND = 300
  AUTO_RESUME = TRUE;

-- Create AICOLLEGE.PUBLIC.TRANSCRIPTS stage to transcripts and semantic yaml file
CREATE OR REPLACE STAGE AICOLLEGE.PUBLIC.TRANSCRIPTS
    DIRECTORY = ( ENABLE = true )
    ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

-- Grant privileges to the AICOLLEGE role
GRANT USAGE, OPERATE ON WAREHOUSE aicollege TO ROLE aicollege;
GRANT ALL ON DATABASE aicollege TO ROLE aicollege;
GRANT ALL ON SCHEMA aicollege.public TO ROLE aicollege;
GRANT CREATE STAGE ON SCHEMA aicollege.public TO ROLE aicollege;
GRANT SELECT ON FUTURE TABLES IN SCHEMA aicollege.public TO ROLE aicollege;
GRANT SELECT ON FUTURE VIEWS  IN SCHEMA aicollege.public TO ROLE aicollege;
GRANT READ ON STAGE AICOLLEGE.PUBLIC.TRANSCRIPTS TO ROLE aicollege;
GRANT DATABASE ROLE SNOWFLAKE.CORTEX_USER TO ROLE aicollege;

-- Grant usage on existing DORA integration and utility database
GRANT USAGE ON INTEGRATION dora_api_integration TO ROLE aicollege;
GRANT USAGE ON DATABASE util_db TO ROLE aicollege;
GRANT USAGE ON SCHEMA util_db.public TO ROLE aicollege;

-- Grant usage on DORA external functions
GRANT USAGE ON FUNCTION util_db.public.se_grader(VARCHAR,BOOLEAN,INTEGER,INTEGER,VARCHAR) TO ROLE aicollege;
GRANT USAGE ON FUNCTION util_db.public.se_greeting(VARCHAR,VARCHAR,VARCHAR,VARCHAR) TO ROLE aicollege;

-- Enable cross-region Cortex inference
ALTER ACCOUNT SET CORTEX_ENABLED_CROSS_REGION = 'AWS_US';

In [ ]:
-- 🔐 Create a PAT (Programmatic Access Token) for Cortex Analyst calls
ALTER USER dataagent_user
  ADD PROGRAMMATIC ACCESS TOKEN AGENT_TOKEN
    ROLE_RESTRICTION = 'AICOLLEGE'
    DAYS_TO_EXPIRY    = 30
    COMMENT           = 'PAT for Data Agent HOL';

-- ⚠️ IMPORTANT: Immediately copy or download the token shown in the message box.
-- This is the ONLY time Snowflake will display it.
-- ❗ If you close the dialog or lose it, you will need to delete and regenerate the token.

#### 🛑 Don't Skip This!
When the token is generated, **copy it immediately** and store it in a secure location like a password manager or secret store (e.g., AWS Secrets Manager, 1Password, etc.).
You’ll need this token to authenticate your Cortex Analyst calls using the dataagent_user.

If you forget to copy it:
```sql
-- Remove and reissue the token if needed
ALTER USER dataagent_user DROP PROGRAMMATIC ACCESS TOKEN AGENT_TOKEN;
-- Then re-run the ADD PROGRAMMATIC ACCESS TOKEN command
```

In [ ]:
-- 🔐 Create key-pair for authenticating your Python process with Snowflake

-- ⚠️ NOTE:
-- You must copy and paste your PUBLIC key into the command below.
-- 👉 The value must include:
--    1️⃣ The line -----BEGIN PUBLIC KEY----- on its own row
--    2️⃣ Your full public key on ONE SINGLE line — no line breaks or spaces
--    3️⃣ The line -----END PUBLIC KEY----- on its own row

-- Replace the 'MIIBIj...' below with your actual public key (keep it as one continuous line)
ALTER USER dataagent_user SET RSA_PUBLIC_KEY='-----BEGIN PUBLIC KEY-----
MIIBIjA...
-----END PUBLIC KEY-----';

-- 📌 Tip: When pasting this into a Snowflake SQL worksheet, ensure the full public key block is preserved exactly as described.
-- Do NOT break the key across multiple lines or insert whitespace — doing so will invalidate the key.

In [ ]:
-- Create Structured Data for Cortex Analyst
USE ROLE AICOLLEGE;
USE DATABASE AICOLLEGE;
USE SCHEMA PUBLIC;
USE WAREHOUSE AICOLLEGE;

-- Create a Customer Meetings Table to track customer meetings, topics discussed, and which Snowflake features were covered. Each meeting links to a customer.
CREATE OR REPLACE TABLE CUSTOMER_MEETINGS (
    MEETING_ID STRING PRIMARY KEY,         
    CUSTOMER_NAME STRING,      
    MEETING_DATE DATE,         
    TOPIC STRING,              
    SNOWFLAKE_FEATURE STRING,  
    MEETING_TYPE STRING,      
    MEETING_DURATION INT,      
    CUSTOMER_SIZE STRING,      
    REGION STRING
);

INSERT INTO CUSTOMER_MEETINGS (
    MEETING_ID, CUSTOMER_NAME, MEETING_DATE, TOPIC, SNOWFLAKE_FEATURE,
    MEETING_TYPE, MEETING_DURATION, CUSTOMER_SIZE, REGION
) VALUES
    /* Acme Corp – audit‑season scaling & autoscaling demo */
    ('M1', 'Acme Corp',        '2025-03-10', 'Audit‑Season Scaling',           'Warehouse Autoscaling',
     'Sales', 30, 'Large',  'North America'),

    /* BetaTech – marketing analytics framework with Snowpark */
    ('M2', 'BetaTech',         '2025-03-12', 'Marketing Analytics',            'Snowpark',
     'Technical Review', 60, 'Medium', 'Europe'),

    /* Gamma LLC – revenue data integration using Document AI */
    ('M3', 'Gamma LLC',        '2025-03-15', 'Revenue Data Integration',       'Document AI',
     'Demo', 45, 'Small',  'Asia'),

    /* Omega Industries – supply‑chain optimization pilot */
    ('M4', 'Omega Industries', '2025-03-17', 'Supply‑Chain Optimization',      'Snowpipe Streaming',
     'Sales', 90, 'Large',  'North America'),

    /* Delta Enterprises – secure Snowpark notebooks framework */
    ('M5', 'Delta Enterprises','2025-03-20', 'Marketing Analytics Framework',  'Snowflake Notebooks',
     'Sales', 30, 'Medium', 'Europe'),

    /* Zeta Solutions – natural‑language analytics with Cortex AI */
    ('M6', 'Zeta Solutions',   '2025-04-01', 'Natural‑Language Analytics',     'Cortex AI NLQ',
     'Technical Review', 50, 'Small',  'North America');

-- Create a Customer Meeting Outcomes Table to tracks the outcome of each meeting (e.g., demo scheduled, deal won, follow-up needed).
CREATE OR REPLACE TABLE CUSTOMER_MEETING_OUTCOMES (
    MEETING_ID STRING PRIMARY KEY,           
    OUTCOME STRING,              
    FOLLOW_UP_DATE DATE,         
    DEAL_VALUE DECIMAL(15,2),    
    CUSTOMER_FEEDBACK STRING,    
    ACTION_REQUIRED STRING,
    FOREIGN KEY(MEETING_ID) REFERENCES CUSTOMER_MEETINGS(MEETING_ID));

INSERT INTO CUSTOMER_MEETING_OUTCOMES VALUES
('M1', 'Demo Scheduled', '2025-03-25', 50000.00, 'Positive', 'Schedule follow-up call'),
('M2', 'POC Planned', '2025-03-22', 0.00, 'Positive', 'Provide architecture deck and Snowpark code samples'),
('M3', 'Follow-up Needed', '2025-04-01', 0.00, 'Neutral', 'Send technical demo'),
('M4', 'Demo Scheduled', '2025-03-22', 75000.00, 'Positive', 'Schedule follow-up call'),
('M5', 'POC Planned', '2025-04-05', 0.00, 'Positive', 'Send pilot charter, sample dataset requirements, security checklist'),
('M6', 'Demo Scheduled', '2025-04-15', 0.00, 'Positive', 'Deliver high‑level Cortex demo and quick‑start ROI outline');

-- Create a Customer Interactions Table to track other interactions with customers, such as support tickets or email communications.
CREATE OR REPLACE TABLE CUSTOMER_INTERACTIONS (
    INTERACTION_ID STRING PRIMARY KEY,       
    CUSTOMER_NAME STRING,       
    INTERACTION_TYPE STRING,     
    FEATURE_DISCUSSED STRING,    
    DATE DATE,                  
    RESOLUTION_STATUS STRING,   
    INTERACTION_DETAIL STRING    
);

INSERT INTO CUSTOMER_INTERACTIONS (
    INTERACTION_ID, CUSTOMER_NAME, INTERACTION_TYPE, FEATURE_DISCUSSED,
    DATE, RESOLUTION_STATUS, INTERACTION_DETAIL
) VALUES
    /* Acme Corp – audit‑busy‑season scaling question */
    ('I1', 'Acme Corp',        'Support Ticket', 'Warehouse Autoscaling',   '2025-03-05',
     'Resolved', 'Provided guidance on multi‑cluster scaling during audit busy season'),

    /* BetaTech – Snowpark ingestion for marketing analytics (matches BetaTech use case) */
    ('I2', 'BetaTech',         'Email',          'Snowpark',                '2025-03-07',
     'Pending',  'Requested code samples for Snowpark ingestion and model deployment'),

    /* Gamma LLC – PDF/image ingestion for revenue reports (Document AI) */
    ('I3', 'Gamma LLC',        'Phone Call',     'Document AI',             '2025-03-09',
     'Resolved', 'Clarified how Document AI extracts tables from partner PDF revenue files'),

    /* Omega Industries – multi‑ERP supply‑chain pilot (Snowpipe & Snowpark) */
    ('I4', 'Omega Industries', 'Support Ticket', 'Snowpipe Streaming',      '2025-03-12',
     'Resolved', 'Helped configure Snowpipe Streaming for SAP ECC change‑data capture'),

    /* Delta Enterprises – secure Snowpark notebooks framework */
    ('I5', 'Delta Enterprises','Meeting',        'Snowflake Notebooks',     '2025-03-16',
     'Pending',  'Scheduled workshop on migrating Jupyter notebooks into Snowflake'),

    /* Zeta Solutions – Cortex AI natural‑language querying demo */
    ('I6', 'Zeta Solutions',   'Email',          'Cortex AI NLQ',           '2025-04-03',
     'Pending',  'Customer asked for NLQ demo assets and quick‑start guide');

-- Create a Customer Industry Table to classify customers by industry (e.g., healthcare, retail, etc.), helping to analyze feature adoption trends across industries.
CREATE OR REPLACE TABLE CUSTOMER_INDUSTRY (
    CUSTOMER_NAME STRING PRIMARY KEY,         
    INDUSTRY STRING,             
    CUSTOMER_SIZE STRING,        
    REGION STRING                
);

INSERT INTO CUSTOMER_INDUSTRY (CUSTOMER_NAME, INDUSTRY, CUSTOMER_SIZE, REGION) VALUES
    ('Acme Corp',        'Professional Services',   'Large',  'North America'),
    ('BetaTech',         'IT Services',             'Medium', 'Europe'),
    ('Gamma LLC',        'Sports Technology',       'Small',  'Asia'),
    ('Omega Industries', 'Industrial Engineering',  'Large',  'North America'),
    ('Delta Enterprises','Consumer Electronics',    'Medium', 'Europe'),
    ('Zeta Solutions',   'Real Estate Technology',  'Small',  'North America');

-- Create a unified view called CUSTOMER_INSIGHTS that merges meetings and interactions into a single analytics layer
CREATE OR REPLACE VIEW CUSTOMER_INSIGHTS
AS (
    -- Core customer meeting details
    SELECT 
        cm.MEETING_ID,
        cm.CUSTOMER_NAME,
        cm.MEETING_DATE,
        cm.TOPIC,
        cm.SNOWFLAKE_FEATURE,
        cm.MEETING_TYPE,
        cmo.OUTCOME,
        cmo.DEAL_VALUE,
        cmo.CUSTOMER_FEEDBACK,
        ci.INDUSTRY,
        ci.REGION
    FROM CUSTOMER_MEETINGS cm
    LEFT JOIN CUSTOMER_MEETING_OUTCOMES cmo ON cm.MEETING_ID = cmo.MEETING_ID
    LEFT JOIN CUSTOMER_INDUSTRY ci ON cm.CUSTOMER_NAME = ci.CUSTOMER_NAME

    UNION ALL

    -- Customer interactions analysis
    SELECT 
        ci.INTERACTION_ID as MEETING_ID,
        ci.CUSTOMER_NAME,
        ci.DATE as MEETING_DATE,
        ci.INTERACTION_DETAIL as TOPIC,
        ci.FEATURE_DISCUSSED as SNOWFLAKE_FEATURE,
        ci.INTERACTION_TYPE as MEETING_TYPE,
        ci.RESOLUTION_STATUS as OUTCOME,
        NULL as DEAL_VALUE,
        NULL as CUSTOMER_FEEDBACK,
        ind.INDUSTRY,
        ind.REGION
    FROM CUSTOMER_INTERACTIONS ci
    LEFT JOIN CUSTOMER_INDUSTRY ind ON ci.CUSTOMER_NAME = ind.CUSTOMER_NAME
);

-- Validate CUSTOMER_INSIGHTS view
SELECT * FROM CUSTOMER_INSIGHTS LIMIT 5;

### Snowflake Intelligence Prerequisites  
*(Run the SQL in the next cells **after Phase 3**.)*

Snowflake Intelligence launches with your user’s **default role** and **default warehouse**.  
The preview also needs a small set of **metadata objects** to store agent definitions.

| What you set up | Why it matters |
|-----------------|----------------|
| **Default role & warehouse** (`AICOLLEGE`) | SI starts under the defaults; if they don’t own your semantic views or search service the UI just spins. |
| **`SNOWFLAKE_INTELLIGENCE` database** | Holds a `TEMP` schema (file uploads) and a `CONFIG` schema for agent metadata. |
| **`AGENTS` schema & `CONFIG` table** | Stores each agent’s name, tools, and visibility. A row-access policy filters rows by role. |

**Quick path**

1. Set your default role to **`AICOLLEGE`** and default warehouse to **`AICOLLEGE`**.  
2. Run the *metadata-DB* script.  
3. Run the *agent-config* script.

In [ ]:
USE ROLE ACCOUNTADMIN;
ALTER USER <YOUR_USERNAME> 
    SET DEFAULT_ROLE = AICOLLEGE, 
        DEFAULT_WAREHOUSE = AICOLLEGE;

### Snowflake Intelligence — Metadata Setup  

The preview needs a small metadata DB so agents have somewhere to keep their config.

**What you need**

* Database called **`SNOWFLAKE_INTELLIGENCE`**  
* Default role **`AICOLLEGE`** (owns everything)  
* Grants for SI users:  
  * `SELECT` on the config tables  
  * `USAGE` on the `TEMP` schema  
  * Access to your Cortex Search Service & any data the agent shows

Run the next SQL to create the DB, `TEMP` schema, and a `CONFIG` schema.  
*(Future builds will do this for you.)*

In [ ]:
-- Create a database. This will hold configuration and other objects to support Snowflake Intelligence.
CREATE DATABASE SNOWFLAKE_INTELLIGENCE;
GRANT OWNERSHIP ON DATABASE SNOWFLAKE_INTELLIGENCE TO ROLE AICOLLEGE;

-- Set up stages and tables for configuration.
USE ROLE AICOLLEGE;
USE DATABASE SNOWFLAKE_INTELLIGENCE;

-- Set up a temp schema for file upload (only temporary stages will be created here).
CREATE OR REPLACE SCHEMA SNOWFLAKE_INTELLIGENCE.TEMP;
GRANT USAGE ON SCHEMA SNOWFLAKE_INTELLIGENCE.TEMP TO ROLE PUBLIC;

-- OPTIONAL: Set up stages and tables for configuration - you can have your semantic models be anywhere else, just make sure that the users have grants to them
CREATE SCHEMA IF NOT EXISTS CONFIG;
GRANT USAGE ON SCHEMA CONFIG TO ROLE AICOLLEGE;
USE SCHEMA CONFIG;

### Agent Metadata Setup  

The preview stores each agent’s config in a user-managed schema.  
The SQL below will:

* create **`SNOWFLAKE_INTELLIGENCE.AGENTS`**  
* add a row-access policy (`AGENT_POLICY`) filtered by `GRANTEE_ROLES`  
* build `AGENTS.CONFIG` (name, tools, sample Qs, etc.)  
* grant **SELECT** to `PUBLIC`, **UPDATE** to `AICOLLEGE`

*(A future release will auto-store agent metadata; for now a dedicated schema lets you secure—or remove—it whenever you like.)*

In [ ]:
USE ROLE AICOLLEGE;

CREATE SCHEMA IF NOT EXISTS SNOWFLAKE_INTELLIGENCE.AGENTS;

-- Make SI agents in general discoverable to everyone.
GRANT USAGE ON SCHEMA SNOWFLAKE_INTELLIGENCE.AGENTS TO ROLE PUBLIC;

CREATE OR REPLACE ROW ACCESS POLICY SNOWFLAKE_INTELLIGENCE.AGENTS.AGENT_POLICY
AS (GRANTEE_ROLES ARRAY)
RETURNS BOOLEAN ->
  ARRAY_SIZE(
    FILTER(
      GRANTEE_ROLES::ARRAY(VARCHAR),
      ROLE -> IS_ROLE_IN_SESSION(ROLE)
    )
  ) <> 0;

-- Create an agent config table. Multiple tables can be created to give granular
-- UPDATE/INSERT permissions to different roles.
CREATE OR REPLACE TABLE SNOWFLAKE_INTELLIGENCE.AGENTS.CONFIG (
    AGENT_NAME            VARCHAR NOT NULL,
    AGENT_DESCRIPTION     VARCHAR,
    GRANTEE_ROLES         ARRAY NOT NULL,
    TOOLS                 ARRAY,
    TOOL_RESOURCES        OBJECT,
    TOOL_CHOICE           OBJECT,
    RESPONSE_INSTRUCTION  VARCHAR,
    SAMPLE_QUESTIONS      ARRAY,
    CONSTRAINT PK_AGENT_NAME PRIMARY KEY (AGENT_NAME)
)
WITH ROW ACCESS POLICY SNOWFLAKE_INTELLIGENCE.AGENTS.AGENT_POLICY
ON (GRANTEE_ROLES);

GRANT SELECT ON TABLE SNOWFLAKE_INTELLIGENCE.AGENTS.CONFIG TO ROLE PUBLIC;
GRANT UPDATE ON TABLE SNOWFLAKE_INTELLIGENCE.AGENTS.CONFIG TO ROLE AICOLLEGE;

#### Quick sanity check
1. **Sign out, then back in** so Snowsight reloads your defaults.  
2. In **Admin ▸ Profile** verify **Role = `AICOLLEGE`** and **Warehouse = `AICOLLEGE`**.  
3. Launch **AI & ML ▸ Snowflake Intelligence** → you should land on the workspace picker (no endless spinner).